In [ ]:
#Bow  Bag of Words  CountVectorizer
# 문서를 고정된 길이의 벡터로 변환
# 문서 - 단어행렬
# 장점 : 간단하고 빠름,
# 단점 : 단어순서 손실, 희소성, 의미적 유사성 무시

# tf-idf TfidfVectorizer
# 모든 문서에서 자주등장하는 단어의 영향을 줄이고 문서 특이 단어를 강조

# multinomal Naive Bayes  확률 모델
# LogisticRegression : 다중클래스  회귀기반 분류

# RidgeClassifer : 회귀 기반 분류 L2규제

# N-gram : 단점 차원폭발에 주의 (정규화/차원 축소 고려)

# Kolnpy Okt


In [3]:
import numpy as np
import pandas as pd
# scikit-learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
#분류모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# # data load
# newsgroups_train =  fetch_20newsgroups(subset='train'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )
# newsgroups_test =  fetch_20newsgroups(subset='test'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )

In [11]:
from sklearn.datasets import load_files
train_path = r'C:\LLM\ch05\20newsbydate\20news-bydate-train'
test_path = r'C:\LLM\ch05\20newsbydate\20news-bydate-test'
newsgroups_train = load_files(train_path,encoding='latin1')
newsgroups_test = load_files(test_path,encoding='latin1')

In [12]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']